In [ ]:
import cv2
import numpy as np

def scan_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply bilateral filtering to reduce noise while preserving edges
    filtered = cv2.bilateralFilter(gray, 9, 75, 75)

    # Apply adaptive thresholding to improve contrast
    scanned = cv2.adaptiveThreshold(
        filtered, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 10
    )

    # Convert back to 3-channel image for drawing the grid
    image_colored = cv2.cvtColor(scanned, cv2.COLOR_GRAY2BGR)

    # Draw a 9x9 black grid
    rows, cols = 9, 9
    cell_height, cell_width = height // rows, width // cols

    for i in range(1, rows):
        cv2.line(image_colored, (0, i * cell_height), (width, i * cell_height), (255, 255, 255), 2)
    for j in range(1, cols):
        cv2.line(image_colored, (j * cell_width, 0), (j * cell_width, height), (255, 255, 255), 2)

    # Draw a border around the whole image
    cv2.rectangle(image_colored, (0, 0), (width - 1, height - 1), (255, 255, 255), 2)

    # Save the processed image
    output_path = image_path.rsplit('.', 1)[0] + "_grid.png"
    cv2.imwrite(output_path, image_colored)
    print(f"Scanned image saved as: {output_path}")

if __name__ == "__main__":
    image_path = "/content/soduko2.jpg"  # Replace with your image file path
    scan_image(image_path)


Scanned image saved as: /content/soduko2_grid.png


In [ ]:
import cv2
import numpy as np
import os
import zipfile
from google.colab import drive

def extract_sudoku_cells(image_path, output_zip="/content/soduko.zip"):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image is loaded properly
    if image is None:
        print("Error: Image not found. Make sure the file path is correct.")
        return

    # Get image dimensions
    height, width, _ = image.shape

    # Define grid cell size
    cell_width = width // 9
    cell_height = height // 9

    # Create a directory to store the cells
    os.makedirs('sudoku_cells', exist_ok=True)

    # Extract each cell
    for row in range(9):
        for col in range(9):
            # Calculate coordinates
            x1 = col * cell_width
            y1 = row * cell_height
            x2 = x1 + cell_width
            y2 = y1 + cell_height

            # Extract the cell (without any grid lines)
            cell = image[y1:y2, x1:x2]

            # Save the cell
            filename = f'sudoku_cells/{row}_{col}.png'
            cv2.imwrite(filename, cell)

    # Create a zip file
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk('sudoku_cells'):
            for file in files:
                zipf.write(os.path.join(root, file), file)

    # Clean up
    for file in os.listdir('sudoku_cells'):
        os.remove(os.path.join('sudoku_cells', file))
    os.rmdir('sudoku_cells')

    print(f"Sudoku cells extracted and saved to {output_zip}")

# Mount Google Drive
#drive.mount('/content/drive')

# Example usage
if __name__ == "__main__":
    # Replace with your image path
    image_path = "/content/soduko2_grid.png"
    extract_sudoku_cells(image_path)


Sudoku cells extracted and saved to /content/soduko.zip


In [ ]:
from google.colab import drive
import zipfile

# Mount Google Drive
#drive.mount('/content/drive')

# Unzip the sudoku_cells.zip file
zip_path = "/content/soduko.zip"
extract_to = "/content/soduko_cells"  # Path where files will be extracted

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Files unzipped to {extract_to}")

Files unzipped to /content/soduko_cells


In [ ]:
!pip install pytranskit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 2.5 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import pickle
import pytranskit
from pytranskit.optrans.continuous.radoncdt import RadonCDT

# Load trained model
model_path = "/content/Rrcdt_ns_model (25).pkl"
with open(model_path, "rb") as f:
    classifier = pickle.load(f)

theta = np.linspace(0, 176, 45)  # Ensure it matches training
radoncdt = RadonCDT(theta)

def preprocess_image(image_path, img_size=28):
    """Load and preprocess an image for prediction"""
    img = Image.open(image_path).convert("L")  # Convert to grayscale
    img = img.resize((img_size, img_size))  # Resize to match training data
    img_array = np.array(img, dtype=np.float32)

    # Ensure white numbers on black background
    if np.mean(img_array) > 127:  # If background is white, invert colors
        img_array = 255 - img_array

    img_array = img_array / 255.0  # Normalize pixel values

    # Ensure input is 3D (N, H, W) to match expected input
    img_array = img_array[np.newaxis, :, :]  # Convert (H, W) -> (1, H, W)

    template = np.ones(img_array[0].shape, dtype=img_array.dtype)

    print(f"Input Shape: {img_array.shape}")  # Debugging Output

    # Apply Radon-CDT transformation
    img_rcdt = radoncdt.forward(
        [0, 1], template / np.sum(template),
        [0, 1], img_array[0] / np.sum(img_array[0]),
        True
    )

    # Return the transformed image, but keep it 3D for compatibility with classifier
    img_rcdt = img_rcdt[np.newaxis, :, :]  # Ensure it's a 3D array (1, H, W)

    return img_array  # Shape: (1, img_size, img_size)
# Function to preprocess and predict a digit

def predict_digit(image_path):
    processed_image = preprocess_image(image_path)

    print(f"Input Shape: {processed_image.shape}")  # Debugging Output

    prediction = classifier.predict(processed_image)  # Use trained classifier
    print(f"Predicted Digit: {prediction[0]}")  # Display prediction
    return prediction[0]

# Load Sudoku from image folder
def load_sudoku(folder_path):
    sudoku_grid = [['x'] * 9 for _ in range(9)]  # Initialize empty 9x9 grid

    for file in os.listdir(folder_path):
        if file.endswith(".png"):
            try:
                row, col = map(int, file[:-4].split("_"))  # Extract row, col from filename
                image_path = os.path.join(folder_path, file)
                digit = predict_digit(image_path)  # Predict digit
                sudoku_grid[row][col] = str(digit)  # Store in grid
            except Exception as e:
                print(f"Error processing {file}: {e}")

    return sudoku_grid

# Print Sudoku in readable format
def print_sudoku(grid):
    for row in grid:
        print(" ".join(row))
    print("\n")

# Sudoku solver using backtracking
def is_valid(board, row, col, num):
    """Check if placing num in (row, col) is valid."""
    num = str(num)

    # Check row
    if num in board[row]:
        return False

    # Check column
    if num in [board[i][col] for i in range(9)]:
        return False

    # Check 3x3 grid
    start_row, start_col = (row // 3) * 3, (col // 3) * 3
    for i in range(3):
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False

    return True

def solve_sudoku(board):
    """Solve the Sudoku puzzle using backtracking."""
    for row in range(9):
        for col in range(9):
            if board[row][col] == 'x':  # Find an empty cell
                for num in range(1, 10):
                    if is_valid(board, row, col, num):
                        board[row][col] = str(num)  # Try placing number
                        if solve_sudoku(board):  # Recur
                            return True
                        board[row][col] = 'x'  # Backtrack
                return False  # No valid number found
    return True  # Solved!

# --- Main Execution ---
sudoku_folder = "/content/soduko_cells"
sudoku_grid = load_sudoku(sudoku_folder)

print("Identified Sudoku:")
print_sudoku(sudoku_grid)

if solve_sudoku(sudoku_grid):
    print("Solved Sudoku:")
    print_sudoku(sudoku_grid)
else:
    print("No solution found.")


Input Shape: (1, 28, 28)
Input Shape: (1, 28, 28)

Calculating RCDTs for testing images ...
Finding nearest subspace for each test sample ...
Predicted Digit: 4
Input Shape: (1, 28, 28)
Error processing 3_6.png: Input contains NaN, infinity or a value too large for dtype('float32').
Input Shape: (1, 28, 28)
Input Shape: (1, 28, 28)

Calculating RCDTs for testing images ...


<ipython-input-12-efaf89f8bc7c>:39: RuntimeWarning: invalid value encountered in divide
  [0, 1], img_array[0] / np.sum(img_array[0]),


Finding nearest subspace for each test sample ...
Predicted Digit: 3
Input Shape: (1, 28, 28)
Error processing 2_5.png: Input contains NaN, infinity or a value too large for dtype('float32').
Input Shape: (1, 28, 28)
Input Shape: (1, 28, 28)

Calculating RCDTs for testing images ...
Finding nearest subspace for each test sample ...
Predicted Digit: 5
Input Shape: (1, 28, 28)
Error processing 7_8.png: Input contains NaN, infinity or a value too large for dtype('float32').
Input Shape: (1, 28, 28)
Error processing 3_4.png: Input contains NaN, infinity or a value too large for dtype('float32').
Input Shape: (1, 28, 28)
Error processing 6_5.png: Input contains NaN, infinity or a value too large for dtype('float32').
Input Shape: (1, 28, 28)
Error processing 6_4.png: Input contains NaN, infinity or a value too large for dtype('float32').
Input Shape: (1, 28, 28)
Error processing 8_7.png: Input contains NaN, infinity or a value too large for dtype('float32').
Input Shape: (1, 28, 28)
Input S